In [1]:
# Load EDA Pkgs
import pandas as pd
import numpy as np

In [2]:

!pip install scikit-multilearn

     |████████████████████████████████| 92kB 7.1MB/s 


In [3]:
# ML Pkgs
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss,classification_report

In [4]:

# Feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# Multi Label Pkgs
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN

In [6]:
Data=pd.read_excel('drive/My Drive/Ankit/october2020 codalab/hindi/constraint_Hindi_Train.xlsx')
Data.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile


#DATA preprocessing

In [7]:
import nltk
import re


def deEmojify(txt):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',txt)
Data['Post']=Data['Post'].apply(lambda x:deEmojify(x))

def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)
Data['Post']=Data['Post'].apply(lambda x:remove_URL(x))


#removing numbers
Data['Post']=Data['Post'].str.replace(r'\d+','')
Data.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील जून को की थी, वो डील ...",non-hostile
3,4,@prabhav साले जेएनयू छाप कमिने लोग हिन्दुओं को...,"defamation,offensive"
4,5,#unlockguidelines - अनलॉक- के लिए गाइडलाइन्स ज...,non-hostile


In [8]:
list(Data['Labels Set'].str.split(',', expand=True).stack().unique())


['hate', 'offensive', 'non-hostile', 'defamation', 'fake']

In [9]:
Data['Labels Set'].str.split(',', expand=True).stack().value_counts()

non-hostile    3050
fake           1144
hate            792
offensive       742
defamation      564
dtype: int64

In [10]:
def Convert(string): 
    li = list(string.split(",")) 
    return li 

str1 =Data['Labels Set'].iloc[0]
print(Convert(str1)) 

['hate', 'offensive']


In [11]:
Data['label']=Data['Labels Set'].apply(lambda x:Convert(x))
Data.head()

,Unique ID,Post,Labels Set,label
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive","[hate, offensive]"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,[non-hostile]
2,3,"सुशांत ने जो बिजनेस डील जून को की थी, वो डील ...",non-hostile,[non-hostile]
3,4,@prabhav साले जेएनयू छाप कमिने लोग हिन्दुओं को...,"defamation,offensive","[defamation, offensive]"
4,5,#unlockguidelines - अनलॉक- के लिए गाइडलाइन्स ज...,non-hostile,[non-hostile]


In [12]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
multilabel= MultiLabelBinarizer()

In [13]:
label=multilabel.fit_transform(Data['label'])
label

array([[0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0]])

In [14]:
Data2=pd.DataFrame(label,columns=multilabel.classes_)
Data2.head()

,defamation,fake,hate,non-hostile,offensive
0,0,0,1,0,1
1,0,0,0,1,0
2,0,0,0,1,0
3,1,0,0,0,1
4,0,0,0,1,0


In [15]:

Data2['defamation']=Data2['defamation'].astype(float)
Data2['fake']=Data2['fake'].astype(float)
Data2['hate']=Data2['hate'].astype(float)
Data2['offensive']=Data2['offensive'].astype(float)

Data2['non-hostile']=Data2['non-hostile'].astype(float)

In [16]:
DATA = pd.concat([Data,Data2.reindex(Data.index)],axis=1)
DATA.head()

,Unique ID,Post,Labels Set,label,defamation,fake,hate,non-hostile,offensive
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive","[hate, offensive]",0.0,0.0,1.0,0.0,1.0
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,[non-hostile],0.0,0.0,0.0,1.0,0.0
2,3,"सुशांत ने जो बिजनेस डील जून को की थी, वो डील ...",non-hostile,[non-hostile],0.0,0.0,0.0,1.0,0.0
3,4,@prabhav साले जेएनयू छाप कमिने लोग हिन्दुओं को...,"defamation,offensive","[defamation, offensive]",1.0,0.0,0.0,0.0,1.0
4,5,#unlockguidelines - अनलॉक- के लिए गाइडलाइन्स ज...,non-hostile,[non-hostile],0.0,0.0,0.0,1.0,0.0


In [17]:
# Build Features
tfidf=TfidfVectorizer()
Xfeatures = tfidf.fit_transform(DATA['Post']).toarray()
Xfeatures

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
y = DATA[['defamation','fake','hate','offensive']]

In [19]:
### Problem Transform
import skmultilearn

In [20]:
dir(skmultilearn)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'adapt',
 'base',
 'problem_transform',
 'utils']

In [21]:
vData=pd.read_excel('drive/My Drive/Ankit/october2020 codalab/hindi/Constraint_Hindi_Valid.xlsx')
vData

,Unique ID,Post,Labels Set
0,1,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए दे...,non-hostile
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,defamation
2,3,कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...,non-hostile
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...,non-hostile
4,5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...",non-hostile
...,...,...,...
806,807,@ThePushpendra_ चंद फेंके हुए टुकड़ों के लिए ह...,hate
807,808,आँखों ही आँखों में इशारा हो गया \nबैठे बैठे जी...,defamation
808,809,औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोन...,non-hostile
809,810,"बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कल...",non-hostile


In [22]:
vData['Post']=vData['Post'].apply(lambda x:deEmojify(x))
vData['Post']=vData['Post'].apply(lambda x:remove_URL(x))
vData['Post']=vData['Post'].str.replace(r'\d+','')
#vData['stemm']=vData['Post'].apply(lambda x:stemm(x))

In [23]:
vData['label']=vData['Labels Set'].apply(lambda x:Convert(x))

In [24]:
vlabel=multilabel.transform(vData['label'])
vlabel

array([[0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0]])

In [25]:
vData2=pd.DataFrame(vlabel,columns=multilabel.classes_)

In [26]:
vData2['defamation']=vData2['defamation'].astype(float)
vData2['fake']=vData2['fake'].astype(float)
vData2['hate']=vData2['hate'].astype(float)
vData2['offensive']=vData2['offensive'].astype(float)

vData2['non-hostile']=vData2['non-hostile'].astype(float)

In [27]:
vDATA = pd.concat([vData,vData2.reindex(vData.index)],axis=1)

# Build Features
t_features1 = tfidf.transform(vDATA['Post'])  
t_vector1=t_features1.toarray()
t_vector1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
vDATA.head()

,Unique ID,Post,Labels Set,label,defamation,fake,hate,non-hostile,offensive
0,1,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए दे...,non-hostile,[non-hostile],0.0,0.0,0.0,1.0,0.0
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,defamation,[defamation],1.0,0.0,0.0,0.0,0.0
2,3,कोरोना से निपटने की तैयारी / दिल्ली में हजार ...,non-hostile,[non-hostile],0.0,0.0,0.0,1.0,0.0
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...,non-hostile,[non-hostile],0.0,0.0,0.0,1.0,0.0
4,5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...",non-hostile,[non-hostile],0.0,0.0,0.0,1.0,0.0


In [29]:
z=vDATA[['defamation','fake','hate','offensive']]

In [30]:
Xfeatures

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
label

array([[0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0]])


# binary classficiation

In [32]:
binary_rel_clf = BinaryRelevance(MultinomialNB())
binary_rel_clf.fit(Xfeatures,label)
# Predictions
br_prediction = binary_rel_clf.predict(t_vector1)
print("binary classficiation")
print("Accuracy score =", accuracy_score(vlabel,br_prediction))
print(metrics.classification_report(vlabel,br_prediction))

binary classficiation
Accuracy score = 0.45129469790382243
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        77
           1       1.00      0.01      0.02       160
           2       0.00      0.00      0.00       110
           3       0.86      0.84      0.85       435
           4       0.00      0.00      0.00       103

   micro avg       0.86      0.41      0.56       885
   macro avg       0.37      0.17      0.17       885
weighted avg       0.60      0.41      0.42       885
 samples avg       0.45      0.45      0.45       885



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Classifier Chains

In [33]:
def build_model(model,mlb_estimator,Xfeatures,y,t_vector1,z):
    # Create an Instance
    clf = mlb_estimator(model)
    clf.fit(Xfeatures,y)
    # Predict
    clf_predictions = clf.predict(t_vector1)
    # Check For Accuracy
    acc = accuracy_score(z,clf_predictions)
    ham = hamming_loss(z,clf_predictions)
    result = {"accuracy:":acc,"hamming_score":ham}
    return result

In [34]:


clf_chain_model = build_model(MultinomialNB(),ClassifierChain,Xfeatures,y,t_vector1,z)

clf_chain_model

{'accuracy:': 0.5388409371146733, 'hamming_score': 0.1381011097410604}